<a href="https://colab.research.google.com/github/pcsilcan/aed/blob/master/20202/aed_20202_0705_more_about_lambdas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%writefile 1.cpp
#include <iostream>
#include <functional>

using namespace std;

function<int()> makeGen(int t1, int t2) {
    int* p1 = new int;
    int* p2 = new int;
    *p1 = t1;
    *p2 = t2;
    return [=]() {
        auto t = *p1;
        *p1 = *p2;
        *p2 = t + *p2;
        return t;
    };
}

int main() {
    auto gen1 = makeGen(0, 1);
    auto gen2 = makeGen(4, 7);
    
    for (int i = 0; i < 10; i++) {
        cout << gen1() << " ";
    }
    cout << endl;
    for (int i = 0; i < 10; i++) {
        cout << gen2() << " ";
    }
    cout << endl;
}


Overwriting 1.cpp


In [8]:
!g++ -std=c++17 1.cpp && ./a.out

0 1 1 2 3 5 8 13 21 34 
4 7 11 18 29 47 76 123 199 322 


In [16]:
%%writefile list.h
#ifndef __LIST_H__
#define __LIST_H__

#include <functional>

using std::function;

template<typename T, typename R=T>
class List {
    struct Node {
        T       element;
        Node*   next;

        Node(T element, Node* next=nullptr): element(element), next(next) {}
    };

    Node*   head;
    int     len;

    function<R(T)> k;
public:
    List(function<R(T)> k=[](T a) { return a; }): head(nullptr), len(0), k(k) {}
    ~List() { destroy(); }
    void add(T element) {
        head = new Node(element, head);
        ++len;
    }
    int findPos(R val) {
        auto aux = head;
        for (int i = 0; i < len; ++i, aux = aux->next) {
            if (k(aux->element) == val) {
                return i;
            }
        }
        return -1;
    }
    T find(R val) {
        auto aux = head;
        for (int i = 0; i < len; ++i, aux = aux->next) {
            if (k(aux->element) == val) {
                return aux->element;
            }
        }
        return 0;
    }
    void process(function<void(T)> proc) {
        auto aux = head;
        for (int i = 0; i < len; ++i, aux = aux->next) {
            proc(aux->element);
        }
    }

private:
    void destroy() {
        while (head != nullptr) {
            auto aux = head;
            head = head->next;
            delete aux;
        }
    }
};

#endif

Overwriting list.h


In [35]:
%%writefile main.cpp
#include <iostream>
#include <string>
#include <time.h>
#include <stdlib.h>

#include "list.h"

using namespace std;

class Goomba {
    int x, y;
public:
    Goomba(int x, int y) : x(x), y(y) {}
    int getX() { return x; }
    int getY() { return y; }
    void setX(int x) { this->x = x; }
    void setY(int y) { this->y = y; }
};

int main() {
    List<Goomba*, int>* l = new List<Goomba*, int>([](Goomba* a) {
        return a->getX();
    });
    srand(time(0));
    for (int i = 0; i < 10; ++i) {
        l->add(new Goomba(rand() % 10, rand() % 10));
    }
    l->process([](Goomba* a) {
        cout << a->getX() << ", " << a->getY() << endl;
    });

    cout << l->findPos(5) << endl;

    
    l->process([](Goomba* a) { delete a; });
    delete l;

    return 0;
}

Overwriting main.cpp


In [40]:
!g++ -std=c++17 -O2 main.cpp && ./a.out

6, 2
3, 7
0, 3
1, 0
7, 1
4, 6
9, 9
7, 8
0, 3
1, 4
-1
